In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 735.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while

In [ ]:
!pip install PyPDF2
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
#Warning controls
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain.llms import OpenAI
import os

os.environ['OPENAI_MODEL_NAME']='gpt-4o-mini'
os.environ["OPENAI_API_KEY"]=""
os.environ["SERPER_API_KEY"]=""

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import BaseTool
import PyPDF2   # For reading PDF files
import docx     # For reading DOCX files
import json
from typing import Dict, List

#Initialize Langchain OpenAI LLM
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
llm = OpenAI(model_name="gpt-4o-mini", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
from crewai_tools import FileReadTool, SerperDevTool, BaseTool

file_read_tool=FileReadTool()
serper_dev_tool=SerperDevTool()

class SentimentAnalysisTool(BaseTool):
    name:str = "SentimentAnalysisTool"
    description:str=("Analyzes the sentiment of text to ensure positive and engaging communication.")

    def _run(self,text:str)->str:
        return "positive"

sentiment_analysis_tool=SentimentAnalysisTool()

In [ ]:
job_description_parser_agent=Agent(
    role="Job Description Parser",
    goal="",
    backstory=(
        "You are an AI-powered job description parser. "
        "Your role is to analyze job descriptions and extract critical information"
        "such as required skills, preferred qualifications, experience levels, and key responsibilities. "
        "You provide a clear and structured breakdown of what the job entails, enabling accurate matching with candidate profiles."
    ),
    allow_delegation=False,
    verbose=True
)

resume_parser_agent=Agent(
    role="Resume Parser",
    goal=(
        "Extract detailed information from resumes, including the candidate's name, skills,"
      "experience, education, certifications, and contact details (email and phone number)."),
    backstory=(
        "You are an AI-powered resume parser with a deep understanding of professional profiles. "
        "Your expertise lies in analyzing resumes and extracting structured data such as the candidate's name, skills, work experience, education, and contact information. "
        "You ensure that no detail is overlooked, providing a comprehensive profile of each candidate for further evaluation."
    ),
    allow_delegation=False,
    verbose=True
)

skill_matcher_agent= Agent(
    role="Skill Matcher",
    goal="Match candidate skills to job requirements and generate a compatibility score.",
    backstory=(
        "You are an AI-powered skill matcher. "
        "Your expertise lies in comparing the skills and qualifications of candidates with the"
        "requirements of job descriptions. You use advanced algorithms to calculate a compatibility score,"
        "ensuring that only the most suitable candidates are shortlisted for further consideration."
    ),
    allow_delegation=False,
    verbose=True
)

candidate_outreach_agent= Agent(
    role="Candidate Outreaching",
    goal="Generate personalized and professional emails for candidate outreach, including the candidate's name, fixed job description details, and assigned interview slot.",
    backstory=(
        "You are an AI-powered email generator. "
        "Your role is to craft personalized and compelling emails for candidates who meet the job requirements. "
        "You highlight their skills and experience, explain why they are a great fit for the role, and include the"
        " candidate's name, fixed job description details, and assigned interview slot that is in future(in March, 2025). "
        "Your emails are professional, engaging, and strictly follow the provided template without adding extra information."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
job_description_parsing_task=Task(
    description=(
        "Parse the job description ({job_description}) and extract key requirements, including"
        "required skills, preferred skills, experience levels, and education requirements. "
        "**Do not add any extra information that is not explicitly mentioned in the job-description.** "
        "This task is essential for understanding the job's demands and ensuring accurate matching with candidate profiles."
    ),
    expected_output=(
        "A JSON object containing the job's required skills, preferred skills, experience requirements,"
        "education requirements, and key responsibilities."
    ),
    agent=job_description_parser_agent
)

resume_parsing_task = Task(
    description=(
        "Parse the candidate's resume ({resume_text}) and extract detailed information, including the"
        "candidate's name, skills, experience, education, certifications, and contact details (email and phone number). "
        "**Do not add any extra information that is not explicitly mentioned in the resume.** "
        "This task is critical for building a comprehensive profile of the candidate, which will be used for skill matching and outreach."
    ),
    expected_output=(
        "A JSON object containing the candidate's name, skills, experience, education, certifications, and contact details (email and phone number). "
        "**Do not include any additional fields or information that is not explicitly present in the resume.**"
    ),
    agent=resume_parser_agent
)

skill_matching_task = Task(
    description=(
        "Compare the candidate's skills and qualifications with the job requirements and generate a compatibility score. "
        "**Do not add any extra information that is not explicitly mentioned in the resume.** "
        "This task ensures that only candidates who meet the job's criteria are shortlisted for further consideration."
    ),
    expected_output=(
        "A compatibility score (0 to 1) indicating how well the candidate's skills match the job requirements."
    ),
    agent=skill_matcher_agent
)

candidate_outreach_task = Task(
    description=(
        "Generate a personalized and professional email for the candidate if their compatibility score is above the threshold. "
        "The email should include:\n"
        "1. The candidate's name (extracted from their resume).\n"
        "2. Fixed job description details (required skills, preferred skills, experience levels, education requirements, and key responsibilities).\n"
        "3. The company's details (name, address, contact email, and phone number).\n"
        "4. A call-to-action to confirm the assigned interview slot( which should be in march, 2025).\n"
        "5. A personalized message highlighting the candidate's relevant skills and experience.\n"
        "**Do not add any extra information that is not explicitly provided in the inputs.**"
    ),
    expected_output=(
        "A personalized email draft tailored to the candidate, including:\n"
        "1. The candidate's name from the resume.\n"
        "2. Fixed job description details.\n"
        "3. The company's details.\n"
        "4. The assigned interview slot in March 2025.\n"
        "5. A personalized message and call-to-action.\n"
        "**Do not include any additional information or fields beyond what is explicitly requested.**"
    ),
    tools=[sentiment_analysis_tool],
    agent=candidate_outreach_agent
)


In [ ]:
#Function to define file reading function

def read_file(file_path):
  if file_path.lower().endswith('.pdf'):
    return read_pdf(file_path)
  elif file_path.lower().endswith('.docx'):
    return read_docx(file_path)
  else:
    raise ValueError(f"Unsupported file type: {file_path}")

def read_pdf(file_path):
  with open(file_path,'rb') as file:
    reader=PyPDF2.PdfReader(file)
    text=""
    for page_num in range (len(reader.pages)):
      page=reader.pages[page_num]
      text+=page.extract_text() if page.extract_text() else ""
    return text

def read_docx(file_path):
  doc=docx.Document (file_path)
  text=""
  for para in doc.paragraphs:
    text+=para.text+"\n"
  return text

In [ ]:
hr_crew=Crew(
    agents=[
        job_description_parser_agent,
        resume_parser_agent,
        skill_matcher_agent,
        candidate_outreach_agent
    ],
    tasks=[
        job_description_parsing_task,
        resume_parsing_task,
        skill_matching_task,
        candidate_outreach_task
    ],
    manager_llm=ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.7),
    verbose=True,
    memory=True
)

if __name__=="__main__":
  #File paths for job description and resumes
  jd_file_path=input("Please enter path to job description: ")
  resume_file_paths=input("Please enter paths to resumes (separated by commas): ").split(',')

  #Read job description
  job_description=read_file(jd_file_path)

  #Process each Resume
  selected_candidates=[]
  unselected_candidates=[]

  for resume_file_path in resume_file_paths:
    #Read Resume
    resume_text=read_file(resume_file_path.strip())

    #Run the crew with inputs
    result = hr_crew.kickoff(
            inputs={
                "job_description": job_description,
                "resume_text": resume_text
            }
        )

    #Ensure result is aprsed as JSON if it is a string
    if isinstance(result,str):
      try:
        result_dict = json.loads(result)
      except json.JSONDecodeError:
        print(f"Error decoding json from result: {result}")
        continue

    elif isinstance(result, dict):
      result_dict=result

    else:
      print(f"Unexpected result type: {type(result)}")
      continue

     # Categorize candidates based on compatibility score
    compatibility_score = result_dict.get("compatibility_score", 0)
    if compatibility_score >= 0.7:  # Threshold for selection
      selected_candidates.append(result_dict)
    else:
      unselected_candidates.append(result_dict)

#Printing Results
print("\n Selected Candidates")
for candidate in selected_candidates:
  print(f"Candidate: {candidate.get('candidate_name', 'N/A')}")
  print(f"Compatibility Score: {candidate.get('compatibility_score', 'N/A')}")
  print(f"Email_draft : {candidate.get('email_draft', 'N/A')}")

print("\n Unselected Candidates")
for candidate in selected_candidates:
  print(f"Candidate: {candidate.get('candidate_name', 'N/A')}")
  print(f"Compatibility Score: {candidate.get('compatibility_score', 'N/A')}")

Please enter path to job description: Sample_JD.pdf
Please enter paths to resumes (separated by commas): Sample_Resume.pdf,Sample_Resume2.pdf
 [DEBUG]: == Working Agent: Job Description Parser
 [INFO]: == Starting Task: Parse the job description (Job Description: Software Developer  
Position:  Software Developer  
Location:  Bengaluru, Karnataka, India  
Employment Type:  Full-Time  
Experience Level:  2-5 years  
Company Name:  Tech Innovators Pvt. Ltd.  
About Us:  We are a fast -growing technology company specializing in innovative software 
solutions for clients across various industries. Our team is passionate about delivering high -
quality, scalable, and efficient applications.  
Key Responsibilities:  
 Design, develop, test, and deploy software applications.  
 Collaborate with cross -functional teams to gather and analyze requirements.  
 Write clean, maintainable, and efficient code following best practices.  
 Troubleshoot, debug, and upgrade  existing software.  
 Pa

Action: SentimentAnalysisTool  
Action Input: {"text": "Subject: Exciting Opportunity at Tech Innovations Inc.\n\nDear Rohan Sharma,\n\nI hope this message finds you well! We are thrilled to inform you that your application for the Software Developer position at Tech Innovations Inc. has been positively received. Your impressive skills in Python, Java, and C++, along with your extensive experience in web application development, make you a strong candidate for our team.\n\n**Job Description Details:**\n- **Required Skills:** Proficiency in Python, Java, C++, HTML, CSS, JavaScript, MySQL, MongoDB, AWS (EC2, S3, Lambda)\n- **Preferred Skills:** Familiarity with Git, Docker, Jenkins, Django, Spring Boot\n- **Experience Level:** 3-5 years in software development\n- **Education Requirements:** Bachelor’s degree in Computer Science or related field\n- **Key Responsibilities:** Develop scalable web applications, collaborate with cross-functional teams, manage database optimization, and implem